# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# import the Data and take a closer look at them

In [4]:
import pickle

def import_and_save_data():
    """ 
    Decription:
    This function loads data from two csv files 
    and saves them in two pickle files (file, file2), 
    in order to shorten the time for loading the data while working in this python notebook
    """
    # load messages dataset 
    azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', delimiter=';')
    customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', delimiter=';')
    
    # Its important to use binary mode 
    file = open('azdias.pickle', 'wb') 
    # source, destination 
    pickle.dump(azdias, file)                      
    file.close() 
    # Its important to use binary mode 
    file2 = open('customers.pickle', 'wb') 
    # source, destination 
    pickle.dump(customers, file2)                      
    file2.close() 

In [5]:
# As it takes about 10-15 minutes to load the origin data, it's better to save them as pickle files
# import_and_save_data()

In [6]:
import time

def load_data(file_name):
    """ 
    Decription:
    This function loads data from two pickle files 
    and saves them into dataframes
    Arguments:
        file_name: pickle file generated in import_and_save_data() function
    Returns:
        df - created dataframe
    """
    s = time.time()
    file = open(file_name,'rb')
    data = pickle.load(file)
    file.close()
    e1 = (time.time() - s)
    print("Time to load: ", (time.time() - s)/60)
    
    return data

In [7]:
# load file (Demographics data for the general population of Germany)
azdias = load_data("data/azdias.pickle")

Time to load:  0.08863286972045899


In [8]:
# load file (Demographics data for customers of a mail-order company)
customers = load_data("data/customers.pickle")

Time to load:  0.019477816422780354


In [9]:
# See the data
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [10]:
# check size of the azdias dataframe
azdias.shape

(891221, 366)

In [25]:
# count Nan values in each column:
for col in azdias.columns:
    nan_values = azdias[col].isnull().sum()
    percent = (nan_values / azdias.shape[0])*100
    if(percent != 0):
        print(col, ', Nan values in %: ', round(percent,2))

AKT_DAT_KL , Nan values in %:  8.25
ALTER_HH , Nan values in %:  8.25
ALTER_KIND1 , Nan values in %:  90.9
ALTER_KIND2 , Nan values in %:  96.69
ALTER_KIND3 , Nan values in %:  99.31
ALTER_KIND4 , Nan values in %:  99.86
ALTERSKATEGORIE_FEIN , Nan values in %:  29.5
ANZ_HAUSHALTE_AKTIV , Nan values in %:  10.45
ANZ_HH_TITEL , Nan values in %:  10.88
ANZ_KINDER , Nan values in %:  8.25
ANZ_PERSONEN , Nan values in %:  8.25
ANZ_STATISTISCHE_HAUSHALTE , Nan values in %:  10.45
ANZ_TITEL , Nan values in %:  8.25
ARBEIT , Nan values in %:  10.91
BALLRAUM , Nan values in %:  10.52
CAMEO_DEU_2015 , Nan values in %:  11.11
CAMEO_DEUG_2015 , Nan values in %:  11.11
CAMEO_INTL_2015 , Nan values in %:  11.11
CJT_GESAMTTYP , Nan values in %:  0.54
CJT_KATALOGNUTZER , Nan values in %:  0.54
CJT_TYP_1 , Nan values in %:  0.54
CJT_TYP_2 , Nan values in %:  0.54
CJT_TYP_3 , Nan values in %:  0.54
CJT_TYP_4 , Nan values in %:  0.54
CJT_TYP_5 , Nan values in %:  0.54
CJT_TYP_6 , Nan values in %:  0.54
D

KBA13_SEG_SONSTIGE , Nan values in %:  11.87
KBA13_SEG_SPORTWAGEN , Nan values in %:  11.87
KBA13_SEG_UTILITIES , Nan values in %:  11.87
KBA13_SEG_VAN , Nan values in %:  11.87
KBA13_SEG_WOHNMOBILE , Nan values in %:  11.87
KBA13_SITZE_4 , Nan values in %:  11.87
KBA13_SITZE_5 , Nan values in %:  11.87
KBA13_SITZE_6 , Nan values in %:  11.87
KBA13_TOYOTA , Nan values in %:  11.87
KBA13_VORB_0 , Nan values in %:  11.87
KBA13_VORB_1 , Nan values in %:  11.87
KBA13_VORB_1_2 , Nan values in %:  11.87
KBA13_VORB_2 , Nan values in %:  11.87
KBA13_VORB_3 , Nan values in %:  11.87
KBA13_VW , Nan values in %:  11.87
KK_KUNDENTYP , Nan values in %:  65.6
KKK , Nan values in %:  13.6
KONSUMNAEHE , Nan values in %:  8.3
KONSUMZELLE , Nan values in %:  10.45
LP_FAMILIE_FEIN , Nan values in %:  0.54
LP_FAMILIE_GROB , Nan values in %:  0.54
LP_LEBENSPHASE_FEIN , Nan values in %:  0.54
LP_LEBENSPHASE_GROB , Nan values in %:  0.54
LP_STATUS_FEIN , Nan values in %:  0.54
LP_STATUS_GROB , Nan values in 

In [12]:
# See the data
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [13]:
# check size of the customers dataframe
customers.shape

(191652, 369)

In [26]:
# count Nan values in each column:
for col in customers.columns:
    nan_values = customers[col].isnull().sum()
    percent = (nan_values / customers.shape[0])*100
    if(percent != 0):
        print(col, ', Nan values in %: ', round(percent,2))

AKT_DAT_KL , Nan values in %:  24.31
ALTER_HH , Nan values in %:  24.31
ALTER_KIND1 , Nan values in %:  93.86
ALTER_KIND2 , Nan values in %:  97.34
ALTER_KIND3 , Nan values in %:  99.33
ALTER_KIND4 , Nan values in %:  99.88
ALTERSKATEGORIE_FEIN , Nan values in %:  27.05
ANZ_HAUSHALTE_AKTIV , Nan values in %:  26.05
ANZ_HH_TITEL , Nan values in %:  27.19
ANZ_KINDER , Nan values in %:  24.31
ANZ_PERSONEN , Nan values in %:  24.31
ANZ_STATISTISCHE_HAUSHALTE , Nan values in %:  26.05
ANZ_TITEL , Nan values in %:  24.31
ARBEIT , Nan values in %:  26.34
BALLRAUM , Nan values in %:  26.07
CAMEO_DEU_2015 , Nan values in %:  26.31
CAMEO_DEUG_2015 , Nan values in %:  26.31
CAMEO_INTL_2015 , Nan values in %:  26.31
CJT_GESAMTTYP , Nan values in %:  1.68
CJT_KATALOGNUTZER , Nan values in %:  1.68
CJT_TYP_1 , Nan values in %:  1.68
CJT_TYP_2 , Nan values in %:  1.68
CJT_TYP_3 , Nan values in %:  1.68
CJT_TYP_4 , Nan values in %:  1.68
CJT_TYP_5 , Nan values in %:  1.68
CJT_TYP_6 , Nan values in %: 

In [34]:
# number of columns differs, here check why:
azdias_col = azdias.columns
customers_col = customers.columns

common_cols = azdias_col.intersection(customers_col)
different_cols = customers_col.difference(azdias_col)
# check common column names
print(common_cols)

Index(['LNR', 'AGER_TYP', 'AKT_DAT_KL', 'ALTER_HH', 'ALTER_KIND1',
       'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'ALTERSKATEGORIE_FEIN',
       'ANZ_HAUSHALTE_AKTIV',
       ...
       'VHN', 'VK_DHT4A', 'VK_DISTANZ', 'VK_ZG11', 'W_KEIT_KIND_HH',
       'WOHNDAUER_2008', 'WOHNLAGE', 'ZABEOTYP', 'ANREDE_KZ',
       'ALTERSKATEGORIE_GROB'],
      dtype='object', length=366)


In [36]:
# check columns that differs:
print(different_cols)

Index(['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], dtype='object')


## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')